In [3]:
import os
import sys
from pathlib import Path
sys.path.insert(1, os.path.realpath(os.path.pardir))


import safetensors
import torch
import torch.nn.functional as F
from accelerate import notebook_launcher
from einops import rearrange
from einops.layers.torch import Rearrange
from simple_parsing import ArgumentParser

from models import brainformer
from utils.data_utils import BrainDataset
from utils.train_utils import TrainConfig, run_train_model, count_parameters




In [4]:
train_config = TrainConfig(exp_name='brainformer_simple', p_augs=0)
data_path = Path(r"D:\Work\brain-to-text-competition\data\competitionData")

train_dataset = BrainDataset(data_path / 'train')
test_dataset = BrainDataset(data_path / 'test')

# Init model
mae_config = brainformer.MAEConfig(window_size=1024)
model = brainformer.MAE(mae_config)
count_parameters(model)

args = (model, (train_dataset, test_dataset), train_config)
notebook_launcher(run_train_model, args, num_processes=1)

Encoder: number of parameters: 8.47M
Shape of casual mask:  torch.Size([8192, 8192])
Shape of the rope cache:  torch.Size([8192, 16])
MAE: number of parameters: 14.78M
MAEConfig(window_size=1024, n_electrodes=256, patch_size=32, dim=256, n_layers=8, head_dim=32, hidden_dim=1024, n_heads=8, n_kv_heads=8, rope_theta=10000, n_dec_layers=4, decoder_dim=256)
Total: 14.78M, Trainable: 14.78M


(14775328, 14775328)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: koval_alvi. Use `wandb login --relogin` to force relogin


Device for training:  cuda
Num devices:  1
Completed initialization of scheduler
*******************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************